#
# Preprocessing and training of the LFADS model
#

This notebook demonstrates how the dataset was preprocessed for LFADS model training, using Session S6 as an example.

Although the saved .pkl.gz file already includes the final LFADS and SpikeCount data, we reconstruct the intermediate steps here for reproducibility and understanding.

**Note:**  
This notebook takes as input a pre-filtered dataset saved in a gzip-compressed pickle file published as the associated dataset to the paper "The geometry of the neural state space of decisions" in [Zenodo](https://zenodo.org/records/15093134). The dataset originates from the raw recordings available at [Zenodo](https://zenodo.org/records/13207505), but includes only completed **decision trials** (`trialType==20`), excluding other tasks or incomplete trials.

After generating the LFADS training dataset, we load the results of running AutoLFADS on NeuroCAAS and augment the DataFrame with two new columns:

- `'SpikeCount'`: Trial-aligned spike count matrix used for training the LFADS model (firing rates in Hz, 10ms bins).
- `'LFADS'`: Smoothed firing rates inferred by AutoLFADS for the same neurons and time windows.

Both columns contain lists of arrays, one per trial, each of shape `(Neurons × Time)`, with time cropped from dot-motion onset to saccade completion — so trial durations vary (not padded with zeros).

##
## Step 0: Set root directory and download some data

In [1]:
import os
import sys

# Automatically find the project root (directory containing 'src' or 'data')
def find_project_root(marker_dirs=("src", "data","notebooks")):
    path = os.getcwd()
    while path != "/" and not all(os.path.exists(os.path.join(path, d)) for d in marker_dirs):
        path = os.path.dirname(path)
    return path

PROJECT_ROOT = find_project_root()

# Set up Python import path and working directory
sys.path.append(os.path.join(PROJECT_ROOT, "src"))
os.chdir(PROJECT_ROOT)

print("Project root:", PROJECT_ROOT)


Project root: /Users/eme/JNotebooks/LIP-Monkey/NeuralGeometry


In [2]:

from io_utils import download_session

# Download and get the path to the file
# Also sets the working session

session = "S6"
path = download_session(session)



File already exists: data/S6.pkl.gz


##
## Step 1: Load preprocessed dataset

In [3]:

from src.io_utils import load_dataframe_with_metadata

df = load_dataframe_with_metadata(session)

print(f"Session: {df.attrs['Session']}, Monkey: {df.attrs['Monkey']}, Date: {df.attrs['Date'].date()}")
print(f"Number of neurons: {df.attrs['NCells']}")
print(f"Columns: {df.columns.tolist()}")
df.head()


Session: S6, Monkey: Jones, Date: 2021-10-11
Number of neurons: 138
Columns: ['fn', 'trial', 'trialType', 'postInj', 'complete', 'fixBreak', 'fixBreakTime', 'direction', 'sDir', 'coh', 'correct', 'incorrect', 'noChoice', 'sCoh', 'choice', 'duration', 'pulseTrial', 'pulseSize', 'pulseDuration', 'pulseDurationReal', 'dotPos', 'dotsDiameter', 'dotsSpeed', 'fixPos', 'fixDiameter', 'fixWindow', 'targ1Pos', 'targ2Pos', 'targ1Diameter', 'targ2Diameter', 'targWindow', 'rewardSize', 'seedBase', 'seedVar', 'trialStart', 'fixOn', 'fixAcquired', 'targetsOn', 'dotsOn', 'dotsOff', 'pulseOn', 'pulseOff', 'saccadeDetected', 'saccadeComplete', 'targetFeedback', 'rewardOn', 'fixOff', 'trialOver', 'spCell', 'unitIdx', 'spCellPop', 'chGood', 'clustGood', 'eye', 'RT', 'LFADS', 'SpikeCount']


,fn,trial,trialType,postInj,complete,fixBreak,fixBreakTime,direction,sDir,coh,...,trialOver,spCell,unitIdx,spCellPop,chGood,clustGood,eye,RT,LFADS,SpikeCount
232,20211011_Jones_2_g0_t1_LIP_chan181_clust173,233,20,0,1,0,NaN,220.0,-1,0.512,...,630.179850,"[628.1694773725, 628.1835436875, 628.210743007...",53,"[[629.1711856625, 629.2569168524999, 629.90885...","[10, 18, 23, 23, 24, 35, 35, 38, 38, 44, 63, 6...","[8, 11, 13, 15, 17, 42, 44, 45, 47, 55, 72, 73...","{'t': [628.1224000000001, 628.1234000000001, 6...",421.0,"[[7.4997506, 7.2157764, 7.238053, 7.041403, 6....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
233,20211011_Jones_2_g0_t1_LIP_chan181_clust173,234,20,0,1,0,NaN,40.0,1,0.512,...,633.379900,633.762154,53,"[[631.5121916666667, 631.7609583333333, 632.57...","[10, 18, 23, 23, 24, 35, 35, 38, 38, 44, 63, 6...","[8, 11, 13, 15, 17, 42, 44, 45, 47, 55, 72, 73...","{'t': [631.4314, 631.4324, 631.4334, 631.43440...",619.0,"[[7.1185827, 6.630899, 6.4970202, 6.4039326, 6...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
234,20211011_Jones_2_g0_t1_LIP_chan181_clust173,235,20,0,1,0,NaN,220.0,-1,0.512,...,636.795875,"[634.8641416666667, 634.8841083333333, 634.886...",53,"[[634.9504416666666, 635.1142083333333, 635.85...","[10, 18, 23, 23, 24, 35, 35, 38, 38, 44, 63, 6...","[8, 11, 13, 15, 17, 42, 44, 45, 47, 55, 72, 73...","{'t': [634.6314000000001, 634.6324000000001, 6...",495.0,"[[7.2089443, 6.4609795, 6.3345013, 6.269701, 5...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
235,20211011_Jones_2_g0_t1_LIP_chan181_clust173,236,20,0,1,0,NaN,40.0,1,0.512,...,640.340900,[],53,"[[638.0602916666667, 638.0836916666667, 638.10...","[10, 18, 23, 23, 24, 35, 35, 38, 38, 44, 63, 6...","[8, 11, 13, 15, 17, 42, 44, 45, 47, 55, 72, 73...","{'t': [638.0474, 638.0484, 638.0494, 638.05040...",442.0,"[[7.083587, 6.598719, 6.432507, 6.347528, 6.04...","[[0.0, 0.0, 100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
236,20211011_Jones_2_g0_t1_LIP_chan181_clust173,237,20,0,1,0,NaN,220.0,-1,0.512,...,643.467900,"[641.7609292475, 641.844093835, 642.5932166666...",53,"[[641.610099685, 641.7770621774999, 641.861526...","[10, 18, 23, 23, 24, 35, 35, 38, 38, 44, 63, 6...","[8, 11, 13, 15, 17, 42, 44, 45, 47, 55, 72, 73...","{'t': [641.5924, 641.5934000000001, 641.594400...",527.0,"[[7.0998855, 6.27805, 6.390462, 6.334432, 6.01...","[[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, ..."


##
## Step 2: Construct training and validation datasets for AutoLFADS

We split the dataset into validation and training, mantaining a choice and coherence balanced split.

In [4]:

import numpy as np
import h5py

coherences = np.sort(df['coh'].unique())
choices = np.sort(df['choice'].unique())
train_idx, valid_idx = [], []

for choice in choices:
    for coh in coherences:
        idx = df[(df['choice'] == choice) & (df['coh'] == coh)].index.tolist()
        np.random.shuffle(idx)
        split = int(2 / 3 * len(idx))
        train_idx.extend(idx[:split])
        valid_idx.extend(idx[split:])

train_idx, valid_idx = np.array(train_idx), np.array(valid_idx)


###
### Handling Variable Trial Durations with External Inputs in LFADS


We create `10ms`-binned spike count data to train the LFADS model. To mitigate artifacts from zero-padding in variable-length reaction time trials, we provide explicit external inputs to the LFADS controller. Each input is a unit step function that remains active for the duration of the trial, one per choice.

This design addresses the challenges posed by the LFADS requirement for uniform trial lengths across the dataset. Without such inputs, shorter trials require significant padding, which can lead to identity overfitting and other training instabilities. By indicating the valid trial window through these step functions, we guide the model to better distinguish meaningful data from padding artifacts.


In [ ]:

bin_size_ms = 10
bin_size_s = bin_size_ms / 1000
max_duration = np.max(df['saccadeComplete'] - df['dotsOn'])
bins = np.arange(0, max_duration + bin_size_s, bin_size_s)
times = bins[:-1] + bin_size_s / 2

def process_trial(trial, df, times, bins, N, choices):
    t0, t1 = df['dotsOn'].loc[trial], df['saccadeComplete'].loc[trial]
    duration = t1 - t0
    spikes = [np.histogram(np.array(sp) - t0, bins=bins)[0] for sp in df['spCellPop'].loc[trial][:N]]
    spike_array = np.stack(spikes, axis=-1)

    valid = times < duration
    current_choice = df['choice'].loc[trial]
    choice_inputs = []

    for ch in choices:
        is_ch = (current_choice == ch)
        active = valid & is_ch & (times > df['dotsOn'].loc[trial] - t0) & (times < df['dotsOff'].loc[trial] - t0)
        choice_inputs.append(active.astype(float))

    inputs = np.stack(choice_inputs, axis=-1)
    return spike_array, inputs


NCells = df.attrs['NCells']
DataT, InputsT = zip(*[process_trial(trial, df, times, bins, NCells, choices) for trial in train_idx])
DataV, InputsV = zip(*[process_trial(trial, df, times, bins, NCells, choices) for trial in valid_idx])

DataT, InputsT = np.stack(DataT), np.stack(InputsT)
DataV, InputsV = np.stack(DataV), np.stack(InputsV)


##
## Step 3: Save HDF5 file for AutoLFADS (NeuroCAAS compatible)

In [ ]:

filename = f"DataLIP_10ms_{session}.h5"
with h5py.File(filename, 'w') as hf:
    hf.create_dataset('train_encod_data', data=DataT)
    hf.create_dataset('valid_encod_data', data=DataV)
    hf.create_dataset('train_recon_data', data=DataT)
    hf.create_dataset('valid_recon_data', data=DataV)
    hf.create_dataset('train_ext_input', data=InputsT)
    hf.create_dataset('valid_ext_input', data=InputsV)
    hf.create_dataset('IndT', data=train_idx)
    hf.create_dataset('IndV', data=valid_idx)

print(f"Saved LFADS dataset to {filename}")


##
## Step 4: Running AutoLFADS on NeuroCAAS

We use the preprocessed dataset `DataLIP_10ms_S6.h5` together with the config file `S6_AutoLFADS.yaml`.
This YAML file specifies all the training parameters, architecture details, and hyperparameter search settings. Files used for all sessions are found in the `data` folder.

NeuroCAAS is an online platform for reproducible cloud-based neuroscience analyses:
Upload the HDF5 data and `S6_AutoLFADS.yaml` configuration to [NeuroCAAS](https://neurocaas.org/analysis/20) for training.
Uploading both files to NeuroCAAS enables reproducible, scalable training of LFADS models across sessions.

📖 How AutoLFADS works:
AutoLFADS (Keshtkaran et al., 2022) is a deep learning framework based on LFADS (Pandarinath et al., 2018) that uses population-based training (PBT) to automatically tune model hyperparameters.
It infers latent dynamical structure from neural spike trains using recurrent networks and variational inference.

**References:**  
- AutoLFADS paper: [Keshtkaran et al., 2022](https://doi.org/10.1038/s41592-022-01675-0)  
- Original LFADS paper: [Pandarinath et al., 2018](https://doi.org/10.1038/s41592-018-0109-9)


##
## Step 5: Load LFADS results and populate DataFrame

In [ ]:

lfads_path = f'./autolfads{session}/lfads_output_{session}.h5'

with h5py.File(lfads_path, 'r') as data:
    IndT, IndV = data['IndT'][:], data['IndV'][:]
    lfads_train, lfads_valid = data['train_output_params'][:], data['valid_output_params'][:]
    data_train, data_valid = data['train_encod_data'][:], data['valid_encod_data'][:]

amp = 1000/bin_size_ms # to get firing rates
df['LFADS'], df['SpikeCount'] = None, None

for i, trial in enumerate(IndT):
    t_end = int(100 * (df['saccadeDetected'].loc[trial] - df['dotsOn'].loc[trial]))
    df.at[trial, 'LFADS'] = (amp * lfads_train[i][:t_end]).T
    df.at[trial, 'SpikeCount'] = (amp * data_train[i][:t_end]).T

for i, trial in enumerate(IndV):
    t_end = int(100 * (df['saccadeDetected'].loc[trial] - df['dotsOn'].loc[trial]))
    df.at[trial, 'LFADS'] = (amp * lfads_valid[i][:t_end]).T
    df.at[trial, 'SpikeCount'] = (amp * data_valid[i][:t_end]).T
